<a href="https://www.kaggle.com/code/vedatgul/titanic-survival-prediction-with-ml?scriptVersionId=123555091" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Loading the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression

In [2]:
# Uploading training data
titanic_train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
titanic_train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
titanic_train_data = titanic_train_data.drop(['Parch', 'SibSp', 'Cabin', 'Name', 'Ticket','PassengerId'], axis=1)
titanic_train_data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
Fare          0
Embarked      2
dtype: int64

In [4]:
# Analyzing data
titanic_train_data.shape

(891, 6)

In [5]:
titanic_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   Fare      891 non-null    float64
 5   Embarked  889 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 41.9+ KB


In [6]:
# Filling in blank lines
titanic_train_data['Age'].fillna(titanic_train_data['Age'].mean(), inplace=True)
titanic_train_data.head()

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0,3,male,22.0,7.2500,S
1,1,1,female,38.0,71.2833,C
2,1,3,female,26.0,7.9250,S
3,1,1,female,35.0,53.1000,S
4,0,3,male,35.0,8.0500,S


In [7]:
print(titanic_train_data['Embarked'].mode()[0])

S


In [8]:
titanic_train_data['Embarked'].fillna(titanic_train_data['Embarked'].mode()[0], inplace=True)

In [9]:
#Converting the categorical features 'Sex' and 'Embarked' into numerical values 0 & 1
titanic_train_data.replace({'Sex': {'male':0, 'female':1}, 'Embarked':{'S':0, 'C':1, 'Q':2}}, inplace=True)
titanic_train_data['Fare'].fillna(titanic_train_data['Fare'].dropna().median(), inplace=True)
titanic_train_data.head()

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0,3,0,22.0,7.2500,0
1,1,1,1,38.0,71.2833,1
2,1,3,1,26.0,7.9250,0
3,1,1,1,35.0,53.1000,0
4,0,3,0,35.0,8.0500,0


In [10]:
titanic_train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.000000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.000000,3.0,3.0000
Sex,891.0,0.352413,0.477990,0.00,0.0000,0.000000,1.0,1.0000
Age,891.0,29.699118,13.002015,0.42,22.0000,29.699118,35.0,80.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.454200,31.0,512.3292
Embarked,891.0,0.361392,0.635673,0.00,0.0000,0.000000,1.0,2.0000


In [11]:
# Uploading test data
titanic_test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
titanic_test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [12]:
titanic_test_data=titanic_test_data.drop(columns=["Name","Ticket","Cabin", 'SibSp', 'Parch'])
titanic_test_data.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked
0,892,3,male,34.5,7.8292,Q
1,893,3,female,47.0,7.0000,S
2,894,2,male,62.0,9.6875,Q
3,895,3,male,27.0,8.6625,S
4,896,3,female,22.0,12.2875,S


In [13]:
titanic_test_data.isnull().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
Fare            1
Embarked        0
dtype: int64

In [14]:
titanic_test_data['Age'].fillna(titanic_test_data['Age'].mean(), inplace=True)
print(titanic_test_data['Embarked'].mode()[0])
titanic_test_data.replace({'Sex': {'male':0, 'female':1}, 'Embarked':{'S':0, 'C':1, 'Q':2}}, inplace=True)
titanic_test_data['Fare'].fillna(titanic_test_data['Fare'].dropna().median(), inplace=True)
titanic_test_data.head()

S


,PassengerId,Pclass,Sex,Age,Fare,Embarked
0,892,3,0,34.5,7.8292,2
1,893,3,1,47.0,7.0000,0
2,894,2,0,62.0,9.6875,2
3,895,3,0,27.0,8.6625,0
4,896,3,1,22.0,12.2875,0


In [15]:
titanic_test_data.shape

(418, 6)

In [16]:
titanic_test_data.isna().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
Fare           0
Embarked       0
dtype: int64

In [17]:
titanic_test_data.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked
0,892,3,0,34.5,7.8292,2
1,893,3,1,47.0,7.0000,0
2,894,2,0,62.0,9.6875,2
3,895,3,0,27.0,8.6625,0
4,896,3,1,22.0,12.2875,0


In [18]:
titanic_train_data.head()

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0,3,0,22.0,7.2500,0
1,1,1,1,38.0,71.2833,1
2,1,3,1,26.0,7.9250,0
3,1,1,1,35.0,53.1000,0
4,0,3,0,35.0,8.0500,0


In [19]:
# Generating X_train, Y_train and X_test data
X_train = titanic_train_data.drop("Survived", axis=1)
Y_train = titanic_train_data["Survived"]
X_test  = titanic_test_data.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 5), (891,), (418, 5))

In [20]:
# Logistic Regression
model = LogisticRegression()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
acc_log = round(model.score(X_train, Y_train) * 100, 2)
acc_log

79.24

In [21]:
submission = pd.DataFrame({
        "PassengerId": titanic_test_data["PassengerId"],
        "Survived": Y_pred
    })

print(submission)

submission.to_csv('submission.csv', index=False)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
